In [2]:
%matplotlib inline
import pandas as pd
import numpy as np

In [3]:
data = pd.read_excel('Template1.xlsx',sheet_name = None)

In [4]:
xls = pd.ExcelFile('Template1.xlsx')
xls.sheet_names 
flow = data.get('Flow vs. Group')
flow.head()

,SN,Flow,Step Name,WS,Area,Step SEQ NO,STEP STATUS,REC TO SCHED?,EXPORT TO CRINV?
0,1,100S PC,5450-21 STI OXIDE BACKSIDE WET ETCH,11-3DNS_SU31_DHF100,F11 WET PROCESS,100,ACTIVE,Y,Y
1,1,100S PC,5450-21 STI OXIDE BACKSIDE WET ETCH,11-3DNS_SU3_DHF100,F11 WET PROCESS,100,ACTIVE,Y,N
2,1,100S PC,5450-21 STI OXIDE BACKSIDE WET ETCH,11-3DNS_SU31_DHF_4,F11 WET PROCESS,100,ACTIVE,Y,N
3,2,100S PC,5200-53 GATE NITRIDE DEP 2 CMP,11-3EBRA_FRXX_BUFF,F11 CMP,200,ACTIVE,Y,N
4,2,100S PC,5200-53 GATE NITRIDE DEP 2 CMP,11-3EBRA_FRXS_BUFF,F11 CMP,200,ACTIVE,Y,Y


In [5]:
process = input('Please Enter the product: ')

Please Enter the product: 120S PC


In [6]:
flow = flow[flow['Flow'] == process]

In [7]:
Qty = data.get('WS vs. Tool ID') 
WS_NAME = Qty['WS']
WS_NAME1 = []
for i in WS_NAME:
    #i.replace('\\xa0', '')
    WS_NAME1.append(i.replace('\xa0', ''))
print(len(WS_NAME1))

3780


In [8]:
G_WS = data.get('Group vs. WS')
G_WS1 = G_WS.set_index(['Capacity Group'],inplace = False)
WS_Group = []
for i in WS_NAME1:
    if i in G_WS1['WS'].tolist():
        group = G_WS1[G_WS1['WS'] ==  i].index.tolist()
    else:
        group = ['0']
    #group = G_WS[G_WS['WS'] ==  i].iat[0,0]
    WS_Group.extend(group)
    #print(group)
Qty['WS Group'] = WS_Group
Qty.head()

,MiCAP ID,WS,Tool ID,Location,WS Group
0,1195118,11-3HIKE_Q2LV_UDPLY,KISDB7VA00,7,11-DIF_UDPLY
1,1195119,11-3HIKE_Q2LV_BSIGE,KISDB7V300,7,11-DIF_BSIGEPLY
2,1195249,11-3TEL_SDRM_EBK,TTSDBC0130,C,11-DRY_SDRM_EBK
3,1195249,11-3TEL_SDRM_EBK,TTSDBC0120,C,11-DRY_SDRM_EBK
4,1195249,11-3TEL_SDRM_EBK,TTSDBC0110,C,11-DRY_SDRM_EBK


In [9]:
step = []
for i in flow['WS']:
    if i in G_WS1['WS'].tolist():
        s = G_WS1[G_WS1['WS'] ==  i].index.tolist()
    else:
        s = ['0']
    step.extend(s)
#print(step.values.tolist())
#print(step)

In [10]:
flow = flow.drop(['Area','Step SEQ NO','STEP STATUS','REC TO SCHED?','EXPORT TO CRINV?'], axis = 1)
flow['WS Group'] = step
flow.head()

,SN,Flow,Step Name,WS,WS Group
3827,1,120S PC,5420-88 PASS NITRIDE DRY ETCH SCRUB,11-3TEL_NS3S_SCRB_DS,11-WET_SCRUB_DS
3828,1,120S PC,5420-88 PASS NITRIDE DRY ETCH SCRUB,11-3DNS_SS33_SCRB_DS,11-WET_SCRUB_DS
3829,1,120S PC,5420-88 PASS NITRIDE DRY ETCH SCRUB,11-3DNS_SS32_SCRB_DS,11-WET_SCRUB_DS
3830,2,120S PC,5420-5C SPACER OXIDE DRY STRIP 2 SCRUB,11-3TEL_NS3S_SCRB_DS,11-WET_SCRUB_DS
3831,2,120S PC,5420-5C SPACER OXIDE DRY STRIP 2 SCRUB,11-3DNS_SS32_SCRB_DS,11-WET_SCRUB_DS


In [11]:
wafer = input('Please enter the quantity of wafers:')
wafer_m = []
for i in step:
    if i in WS_Group:
        w = round(int(wafer)/Qty.groupby(['WS Group']).size()[i],2)
        wafer_m.append(w)
    else:
        wafer_m.append(0)
    #print(i)
print ('Number of wafers in each machine:',wafer_m)

Please enter the quantity of wafers:7000
Number of wafers in each machine: [218.75, 218.75, 218.75, 218.75, 218.75, 218.75, 218.75, 218.75, 218.75, 218.75, 218.75, 218.75, 218.75, 218.75, 218.75, 700.0, 700.0, 700.0, 700.0, 1000.0, 1000.0, 194.44, 218.75, 218.75, 3500.0, 3500.0, 3500.0, 218.75, 218.75, 218.75, 777.78, 777.78, 777.78, 388.89, 1750.0, 1400.0, 218.75, 218.75, 218.75, 7000.0, 318.18, 318.18, 1000.0, 218.75, 218.75, 218.75, 2333.33, 636.36, 388.89, 3500.0, 1400.0, 1750.0, 1400.0, 250.0, 0, 0, 291.67, 291.67, 291.67, 1750.0, 259.26, 1400.0, 1400.0, 1750.0, 538.46, 250.0, 0, 0, 116.67, 116.67, 259.26, 259.26, 218.75, 194.44, 218.75, 218.75, 218.75, 388.89, 875.0, 269.23, 0, 140.0, 388.89, 3500.0, 1400.0, 1750.0, 388.89, 875.0, 1166.67, 269.23, 875.0, 0, 140.0, 20.17, 1166.67, 0, 318.18, 318.18, 318.18, 318.18, 318.18, 318.18, 318.18, 318.18, 0, 140.0, 20.17, 1166.67, 0, 259.26, 259.26, 1400.0, 1400.0, 259.26, 700.0, 700.0, 1000.0, 1000.0, 700.0, 700.0, 116.67, 116.67, 700.0, 

In [12]:
#Qty[(Qty['WS Group'] == '11-CVD_AC_SR') & (Qty['Location'] == 'D')].shape[0]

In [13]:
col = ['3','4','7','8','9','C','D','E']
#print(wafer_in_each_machine)
wafer_in_each_machine = pd.DataFrame(columns = col)
machine_num = pd.DataFrame(columns = col)
for i in step:
    f1 = Qty[(Qty['WS Group'] == i) & (Qty['Location'] == '3')].shape[0]
    f2 = Qty[(Qty['WS Group'] == i) & (Qty['Location'] == '4')].shape[0]
    f3 = Qty[(Qty['WS Group'] == i) & (Qty['Location'] == '7')].shape[0]
    f4 = Qty[(Qty['WS Group'] == i) & (Qty['Location'] == '8')].shape[0]
    f5 = Qty[(Qty['WS Group'] == i) & (Qty['Location'] == '9')].shape[0]
    f6 = Qty[(Qty['WS Group'] == i) & (Qty['Location'] == 'C')].shape[0]
    f7 = Qty[(Qty['WS Group'] == i) & (Qty['Location'] == 'D')].shape[0]
    f8 = Qty[(Qty['WS Group'] == i) & (Qty['Location'] == 'E')].shape[0]
    machine = np.array([f1,f2,f3,f4,f5,f6,f7,f8])
    #print(machine)
    machine_D = pd.DataFrame([machine],columns = col)
    machine_num = machine_num.append(machine_D)
    for j in range(8):
        if i in WS_Group:
            num = wafer_m[step.index(i)]*machine[j]
        else:
            num = 0
        machine[j] = num
    machine = pd.DataFrame([machine],columns = col)
    #print(machine)
    wafer_in_each_machine = wafer_in_each_machine.append(machine)
print(wafer_in_each_machine)

       3    4     7     8     9     C    D    E
0   1531    0  2843     0   218  2187  218    0
0   1531    0  2843     0   218  2187  218    0
0   1531    0  2843     0   218  2187  218    0
0   1531    0  2843     0   218  2187  218    0
0   1531    0  2843     0   218  2187  218    0
0   1531    0  2843     0   218  2187  218    0
0   1531    0  2843     0   218  2187  218    0
0   1531    0  2843     0   218  2187  218    0
0   1531    0  2843     0   218  2187  218    0
0   1531    0  2843     0   218  2187  218    0
0   1531    0  2843     0   218  2187  218    0
0   1531    0  2843     0   218  2187  218    0
0   1531    0  2843     0   218  2187  218    0
0   1531    0  2843     0   218  2187  218    0
0   1531    0  2843     0   218  2187  218    0
0   4900    0  1400     0     0   700    0    0
0   4900    0  1400     0     0   700    0    0
0   4900    0  1400     0     0   700    0    0
0   4900    0  1400     0     0   700    0    0
0   3000    0  4000     0     0     0   

In [14]:
wafer_in_each_machine['WS Group'] = step 
wafer_in_each_machine.set_index('WS Group',inplace = True)
wafer_in_each_machine.head(20)

,3,4,7,8,9,C,D,E
WS Group,,,,,,,,
11-WET_SCRUB_DS,1531,0,2843,0,218,2187,218,0
11-WET_SCRUB_DS,1531,0,2843,0,218,2187,218,0
11-WET_SCRUB_DS,1531,0,2843,0,218,2187,218,0
11-WET_SCRUB_DS,1531,0,2843,0,218,2187,218,0
11-WET_SCRUB_DS,1531,0,2843,0,218,2187,218,0
11-WET_SCRUB_DS,1531,0,2843,0,218,2187,218,0
11-WET_SCRUB_DS,1531,0,2843,0,218,2187,218,0
11-WET_SCRUB_DS,1531,0,2843,0,218,2187,218,0
11-WET_SCRUB_DS,1531,0,2843,0,218,2187,218,0


In [15]:
wafer_in_each_machine.shape
w = wafer_in_each_machine.values
#print(w)
w.shape

(2003, 8)

In [16]:
import itertools
col1 = ['3','4','7','8','9','C','D','E']
print(list(itertools.product(col, col1)))

[('3', '3'), ('3', '4'), ('3', '7'), ('3', '8'), ('3', '9'), ('3', 'C'), ('3', 'D'), ('3', 'E'), ('4', '3'), ('4', '4'), ('4', '7'), ('4', '8'), ('4', '9'), ('4', 'C'), ('4', 'D'), ('4', 'E'), ('7', '3'), ('7', '4'), ('7', '7'), ('7', '8'), ('7', '9'), ('7', 'C'), ('7', 'D'), ('7', 'E'), ('8', '3'), ('8', '4'), ('8', '7'), ('8', '8'), ('8', '9'), ('8', 'C'), ('8', 'D'), ('8', 'E'), ('9', '3'), ('9', '4'), ('9', '7'), ('9', '8'), ('9', '9'), ('9', 'C'), ('9', 'D'), ('9', 'E'), ('C', '3'), ('C', '4'), ('C', '7'), ('C', '8'), ('C', '9'), ('C', 'C'), ('C', 'D'), ('C', 'E'), ('D', '3'), ('D', '4'), ('D', '7'), ('D', '8'), ('D', '9'), ('D', 'C'), ('D', 'D'), ('D', 'E'), ('E', '3'), ('E', '4'), ('E', '7'), ('E', '8'), ('E', '9'), ('E', 'C'), ('E', 'D'), ('E', 'E')]


In [17]:
a = range(8)
m = []
for k in range(len(step)-1):
    for j in a:
        for i in a:
            n = max(w[k,j]-w[k+1,j],0)*w[k+1,a]//(int(wafer)-w[k+1,j]+1) #避免分母為零
        m.append(n)      
step_m = np.array(m)
#print(step_m)
#print(step_m.shape)
carry = np.vsplit(step_m, len(step)-1)
for i in range(len(step)-1):
    for j in range(8):
        carry[i][j][j] = min(w[i,j],w[i+1,j])
        carry = np.array(carry)
#print(carry)

In [18]:
def weight(distance,carry,step):
    carry_amount = pd.DataFrame(columns = list(itertools.product(col, col1)))
    total_carry = []
    step_carriage = []
    for i in range(len(step)-1):
        carry_weight = np.multiply(carry[i],distance)
        total_carry_step = np.sum(carry_weight)-np.trace(carry_weight)
        total_carry.append(total_carry_step)
        carry_weight = carry_weight.reshape(1,len(col)**2)
        carry_weight = pd.DataFrame(carry_weight,columns = list(itertools.product(col, col1)))
        carry_amount = carry_amount.append(carry_weight)
    #carry_amount = pd.concat([pd.DataFrame(step[1:len(step)],columns = ['To']),carry_amount],axis = 1)
    #carry_amount = pd.concat([pd.DataFrame(step[:len(step)-1],columns = ['From']),carry_amount],axis = 1)
    #carry_amount.insert(0,'From',step[:len(step)-1])
    #carry_amount.insert(1,'To',step[1:len(step)])
    carry_amount.index = range(len(carry_amount))
    carry_amount['Step carriage'] = total_carry
    carry_amount.loc['Fab Carriage'] = carry_amount.sum(axis = 0)
    #print ('Total carriage:',sum(total_carry))
    carry_amount.index = range(len(carry_amount))
    return carry_amount
weight_distance = np.array([[0,1,1,2,3,1,2,3],[1,0,2,3,4,2,3,4],[1,2,0,1,2,1,2,3],[2,3,1,0,1,2,3,4]
                           ,[3,4,2,1,0,3,4,5],[1,2,1,2,3,0,1,2],[2,3,2,3,4,1,0,1],[3,4,3,4,5,2,1,0]])
weight(distance = weight_distance,carry = carry,step = step).head(100)

,"(3, 3)","(3, 4)","(3, 7)","(3, 8)","(3, 9)","(3, C)","(3, D)","(3, E)","(4, 3)","(4, 4)",...,"(D, E)","(E, 3)","(E, 4)","(E, 7)","(E, 8)","(E, 9)","(E, C)","(E, D)","(E, E)",Step carriage
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0


In [19]:
weight(distance = weight_distance,carry = carry,step = step).to_excel('template_result for 120S PC 7K.xlsx') 

PermissionError: [Errno 13] Permission denied: 'template_result for 120S PC 7K.xlsx'